In [1]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [2]:
from runs.models import Run
from run_histos.models import RunHisto

In [3]:
import pandas as pd
import numpy as np
import altair as alt

## Checking apps

In [4]:
runs = Run.objects.all().values()

In [5]:
runs[0]

{'id': 1,
 'run_number': 315258,
 'date': datetime.datetime(2021, 10, 26, 11, 37, 39, 211844, tzinfo=<UTC>)}

In [6]:
run_histos = RunHisto.objects.all().values()

In [7]:
run_histos[0]

{'id': 1,
 'run_id': 1,
 'date': datetime.datetime(2021, 10, 26, 11, 37, 48, 578496, tzinfo=<UTC>),
 'primary_dataset': 'EGamma',
 'path': 'TrackParameters/generalTracks/GeneralProperties',
 'title': 'AbsDistanceOfClosestApproachToBS_GenTk',
 'entries': 786498,
 'mean': 0.4792563974038401,
 'rms': 2.9386499655611944,
 'skewness': 2.917119692221068,
 'kurtosis': 29.826709167508028}

## Converting to dataframes

In [8]:
df = pd.DataFrame(run_histos)

In [9]:
df.head()

,id,run_id,date,primary_dataset,path,title,entries,mean,rms,skewness,kurtosis
0,1,1,2021-10-26 11:37:48.578496+00:00,EGamma,TrackParameters/generalTracks/GeneralProperties,AbsDistanceOfClosestApproachToBS_GenTk,786498,0.479256,2.938650,2.917120,29.826709
1,2,1,2021-10-26 11:37:48.578595+00:00,EGamma,TrackParameters/generalTracks/GeneralProperties,Chi2Prob_GenTk,786498,0.492466,0.322329,-0.069946,-1.345512
2,3,1,2021-10-26 11:37:48.578635+00:00,EGamma,TrackParameters/generalTracks/GeneralProperties,Chi2_GenTk,786498,19.083402,68.698831,0.043568,-2.917017
3,4,1,2021-10-26 11:37:48.578670+00:00,EGamma,TrackParameters/generalTracks/GeneralProperties,Chi2oNDFVsEta_ImpactPoint_GenTk,786288,-0.106133,1.589274,0.272494,-0.112022
4,5,1,2021-10-26 11:37:48.578703+00:00,EGamma,TrackParameters/generalTracks/GeneralProperties,Chi2oNDFVsNHits_ImpactPoint_GenTk,786288,11.331133,6.684785,37.555632,171.534894


In [10]:
df.shape

(17468, 11)

## Filtering

In [11]:
select_title = ['Chi2Prob_GenTk', 'Chi2_GenTk']

In [12]:
df_filtered = df[df['title'].isin(select_title)][['primary_dataset', 'run_id', 'title', 'mean']]

In [13]:
df_filtered

,primary_dataset,run_id,title,mean
1,EGamma,1,Chi2Prob_GenTk,0.492466
2,EGamma,1,Chi2_GenTk,19.083402
84,EGamma,2,Chi2Prob_GenTk,0.486288
85,EGamma,2,Chi2_GenTk,19.146579
167,EGamma,3,Chi2Prob_GenTk,0.499791
...,...,...,...,...
17182,ZeroBias,49,Chi2_GenTk,19.138309
17277,ZeroBias,50,Chi2Prob_GenTk,0.505047
17278,ZeroBias,50,Chi2_GenTk,19.139480
17373,ZeroBias,51,Chi2Prob_GenTk,0.516412


In [14]:
df_filtered = df_filtered.pivot(index=['primary_dataset', 'run_id'], columns='title', values='mean').reset_index()

In [15]:
df_filtered

title,primary_dataset,run_id,Chi2Prob_GenTk,Chi2_GenTk
0,EGamma,1,0.492466,19.083402
1,EGamma,2,0.486288,19.146579
2,EGamma,3,0.499791,19.027085
3,EGamma,4,0.489095,19.103232
4,EGamma,5,0.483947,19.190387
...,...,...,...,...
197,ZeroBias,51,0.516412,19.086274
198,ZeroBias,52,0.497623,18.862586
199,ZeroBias,53,0.502357,18.838694
200,ZeroBias,54,0.479684,19.094168


In [16]:
df_filtered.primary_dataset.unique()

array(['EGamma', 'JetHT', 'SingleMuon', 'ZeroBias'], dtype=object)

## Testing cross-filtering with altair

In [17]:
source = df_filtered

In [18]:
brush_EGamma     = alt.selection(type='interval')
brush_SingleMuon = alt.selection(type='interval')
brush_JetHT      = alt.selection(type='interval')
brush_ZeroBias   = alt.selection(type='interval')

In [19]:
EGamma = alt.Chart(source[source['primary_dataset']=='EGamma']).mark_point(clip=True).encode(
    alt.X('Chi2Prob_GenTk:Q',
        scale=alt.Scale(domain=(0.4, 0.6))
    ),
    alt.Y('Chi2_GenTk:Q',
        scale=alt.Scale(domain=(16, 22))
    ),
    color=alt.condition(brush_EGamma|brush_SingleMuon|brush_JetHT|brush_ZeroBias, 'primary_dataset', alt.value('lightgray'))
).add_selection(
    brush_EGamma
)

In [20]:
SingleMuon = alt.Chart(source[source['primary_dataset']=='SingleMuon']).mark_point(clip=True).encode(
    alt.X('Chi2Prob_GenTk:Q',
        scale=alt.Scale(domain=(0.4, 0.6))
    ),
    alt.Y('Chi2_GenTk:Q',
        scale=alt.Scale(domain=(16, 22))
    ),
    color=alt.condition(brush_EGamma|brush_SingleMuon|brush_JetHT|brush_ZeroBias, 'primary_dataset', alt.value('lightgray'))
).add_selection(
    brush_SingleMuon
)

In [21]:
JetHT = alt.Chart(source[source['primary_dataset']=='JetHT']).mark_point(clip=True).encode(
    alt.X('Chi2Prob_GenTk:Q',
        scale=alt.Scale(domain=(0.4, 0.6))
    ),
    alt.Y('Chi2_GenTk:Q',
        scale=alt.Scale(domain=(16, 22))
    ),
    color=alt.condition(brush_EGamma|brush_SingleMuon|brush_JetHT|brush_ZeroBias, 'primary_dataset', alt.value('lightgray'))
).add_selection(
    brush_JetHT
)

In [22]:
ZeroBias = alt.Chart(source[source['primary_dataset']=='ZeroBias']).mark_point(clip=True).encode(
    alt.X('Chi2Prob_GenTk:Q',
        scale=alt.Scale(domain=(0.4, 0.6))
    ),
    alt.Y('Chi2_GenTk:Q',
        scale=alt.Scale(domain=(16, 22))
    ),
    color=alt.condition(brush_EGamma|brush_SingleMuon|brush_JetHT|brush_ZeroBias, 'primary_dataset', alt.value('lightgray'))
).add_selection(
    brush_ZeroBias
)

In [24]:
EGamma & SingleMuon & JetHT & ZeroBias

alt.VConcatChart(...)

In [27]:
# Add table
# https://altair-viz.github.io/gallery/scatter_linked_table.html